Message schema {

   **datetime** timestamp; // ISO 8601 format in UTC

   **float** FM; // flow metering  (cc per minute: ccm) 유량 츨정


   **float** NOX; // nitrogen oxide (parts per million: ppm) 질소 산화물
   
   **float** O2; // oxygen (ppm)

   **float** temp; // air temperature (F’)
   
}

1. csv 데이터를 가져오기 및 데이터를 메모리로 불러오기.
 - 데이터의 수는 1000개 이상이어야 한다
 - descriptive statistics 및 histogram을 적용하여 데이터셋의 정보를 파악해야 한다.
 - 값이 없는 경우(Nan, Null), 전후 값의 평균치를 넣는다.

In [4]:
import numpy as np
import pandas as pd


In [5]:
data = pd.read_csv("/content/sample_data/factory_air_pollution_dataset.csv")

In [6]:
data

,timestamp,FM,NOX,O2,temp
0,2022-03-01 00:00:00+00:00,2050.36,36.75,5.60,98.38
1,2022-03-01 00:05:00+00:00,2038.29,36.65,5.64,98.30
2,2022-03-01 00:10:00+00:00,2051.85,36.27,5.60,98.28
3,2022-03-01 00:15:00+00:00,2043.45,36.51,5.64,98.29
4,2022-03-01 00:20:00+00:00,2071.23,36.68,5.61,98.30
...,...,...,...,...,...
8921,2022-03-31 23:35:00+00:00,1788.09,38.38,5.89,91.50
8922,2022-03-31 23:40:00+00:00,1763.63,38.34,5.88,91.47
8923,2022-03-31 23:45:00+00:00,1762.03,38.65,5.99,91.45
8924,2022-03-31 23:50:00+00:00,1826.19,39.04,6.06,91.50


In [7]:
data.describe()

,FM,NOX,O2,temp
count,8926.000000,8922.000000,8926.000000,8860.000000
mean,1543.517650,35.579015,8.576594,91.462454
std,589.796767,11.413413,3.870188,14.176795
min,5.750000,0.000000,0.090000,28.490000
25%,1264.572500,35.662500,6.140000,91.420000
50%,1745.960000,38.330000,7.280000,94.500000
75%,1891.577500,40.430000,9.510000,98.580000
max,4793.210000,401.430000,20.830000,106.580000


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8926 entries, 0 to 8925
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  8926 non-null   object 
 1   FM         8926 non-null   float64
 2   NOX        8922 non-null   float64
 3   O2         8926 non-null   float64
 4   temp       8860 non-null   float64
dtypes: float64(4), object(1)
memory usage: 348.8+ KB


In [11]:
data.isnull().value_counts()

timestamp  FM     NOX    O2     temp 
False      False  False  False  False    8856
                                True       66
                  True   False  False       4
dtype: int64

In [13]:
data.isnull().sum()

timestamp     0
FM            0
NOX           4
O2            0
temp         66
dtype: int64

In [19]:
data.fillna(data.mean())

<ipython-input-19-f4696802a847>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.fillna(data.mean())


,timestamp,FM,NOX,O2,temp
0,2022-03-01 00:00:00+00:00,2050.36,36.75,5.60,98.38
1,2022-03-01 00:05:00+00:00,2038.29,36.65,5.64,98.30
2,2022-03-01 00:10:00+00:00,2051.85,36.27,5.60,98.28
3,2022-03-01 00:15:00+00:00,2043.45,36.51,5.64,98.29
4,2022-03-01 00:20:00+00:00,2071.23,36.68,5.61,98.30
...,...,...,...,...,...
8921,2022-03-31 23:35:00+00:00,1788.09,38.38,5.89,91.50
8922,2022-03-31 23:40:00+00:00,1763.63,38.34,5.88,91.47
8923,2022-03-31 23:45:00+00:00,1762.03,38.65,5.99,91.45
8924,2022-03-31 23:50:00+00:00,1826.19,39.04,6.06,91.50


In [20]:
data.isnull().sum()

timestamp     0
FM            0
NOX           4
O2            0
temp         66
dtype: int64

2. Datatime의 period 기능을 사용하여 주어진 데이터를 7일단위로 그룹화한다.

3. 특정 컬럼(columns)에 대해 10개, 100개, 1000개 bins을 나누고 Probability Mass function(PMF)를 구하여라
 - pd_Series.value_counts(normalize=True, bins=10)를 사용하면 구성할 수 있다.


4. 세 개의 임의 컬럼 x, y, z이 PMF px, py, pz로 변환되었다고 가정하다. 이때 결합분포(Joint distribution) pxyz를 구하여라.
 - 중첩 for-loop를 사용하여 구하시오.
 - chaintools나 itertools 라이브러리를 사용하여 pxyz를 구하시오.


5. px의 누적함수분포(CDF) cx를 구하시오
 - itertools의 acculmulate 함수를 이용하여 cx를 계산하시오. (이때, pmf는 오름차 순으로 이미 정렬되어 있어야 한다